# Tiền xử lí dữ liệu - Preprocessing data

In [52]:
import pandas as pd
import numpy as np

In [53]:
df = pd.read_csv('data.csv')
df.head()

,cum,sbd,toan,ngu_van,ngoai_ngu,vat_li,hoa_hoc,sinh_hoc,lich_su,dia_ly,gdcd
0,1,1000001,3.6,5.00,4.0,NaN,NaN,NaN,2.75,6.0,8.75
1,1,1000002,8.4,6.75,7.6,NaN,NaN,NaN,8.50,7.5,8.25
2,1,1000003,5.8,7.50,5.0,NaN,NaN,NaN,7.25,5.5,8.75
3,1,1000004,7.4,7.50,8.6,NaN,NaN,NaN,7.50,6.5,7.50
4,1,1000005,7.2,8.50,9.0,NaN,NaN,NaN,8.00,8.5,8.25


### **Nhận xét:**
Cơ bản sẽ có 4 đối tượng thi:
- **1. Khóa 2022 thi THPTQG:**

    - Khối khoa học tự nhiến: Toán Văn Anh Lý Hóa Sinh --- 6 môn
    - Khôi khoc học xã hội : Toán Văn Anh Sử Địa GDCD --- 6 môn  
    
- **2. Giáo dục thường xuyên xét tốt nghiệp + đại học:**

    - Thi 3 môn tổ hợp khtn hoặc khxh
    - Không thi tiếng anh
    - Tổng thi: 5 môn
    - VD: KHTN: Toán Văn  Lý Hóa Sinh
    - VD: KHXH: Toán Văn  Sử Địa GDCD

- **3. Khóa 2021 về trước thi lại đại học (Đã tốt nghiệp THPT)**

    - Thi một khối duy nhất (Vd: khối A - Toán Lý Hóa)         --- 3  môn
    - Thi nhiều hơn một khối (Vd: khối A B - Toán Lý Hóa Sinh) --- 3 < số môn < 6

- **4. Khóa 2021 về trước thi lại tốt nghiệp (Chưa tốt nghiệp):**

    - Một số môn năm trước đủ cao và đã bảo lưu
    - Chỉ thi những môn còn thiếu để xét tốt nghiệp
    - Nếu thi >= 3 môn thì có thể lấy điểm 3 môn đó xét điểm ĐH nên đưa vào trường hợp phía trên (TH3)
    - Chỉ thi 1 - 2 môn
    

### **1. Kiểm tra kiểu dữ liệu**

In [54]:
print(df.dtypes)

cum            int64
sbd            int64
toan         float64
ngu_van      float64
ngoai_ngu    float64
vat_li       float64
hoa_hoc      float64
sinh_hoc     float64
lich_su      float64
dia_ly       float64
gdcd         float64
dtype: object


### **2. Xác định các trường hợp xảy ra**

In [55]:
df.count()

cum          995441
sbd          995441
toan         982726
ngu_van      981407
ngoai_ngu    870609
vat_li       325523
hoa_hoc      327367
sinh_hoc     322198
lich_su      659662
dia_ly       657421
gdcd         554343
dtype: int64

*Có thể thấy số lượng thí sinh của từng môn đề khác nhau. Điều này xảy ra là do những thí sinh thi tự do và chỉ thi những môn mà mình mong muốn.*

In [56]:
isNan_score,  isNan_counts = np.unique(np.isnan(df.loc[:, 'toan':]), return_counts=True, axis=0)
num_type_score = len(isNan_counts)
for i in range (num_type_score):
    print(isNan_score[i], ' --> ', isNan_counts[i])


[False False False False False False  True  True  True]  -->  309162
[False False False False False  True  True  True  True]  -->  105
[False False False False  True False  True  True  True]  -->  4
[False False False False  True  True  True  True  True]  -->  176
[False False False  True False False  True  True  True]  -->  47
[False False False  True False  True  True  True  True]  -->  54
[False False False  True  True False  True  True  True]  -->  8
[False False False  True  True  True False False False]  -->  549574
[False False False  True  True  True False False  True]  -->  1804
[False False False  True  True  True False  True False]  -->  40
[False False False  True  True  True False  True  True]  -->  257
[False False False  True  True  True  True False False]  -->  30
[False False False  True  True  True  True False  True]  -->  104
[False False False  True  True  True  True  True False]  -->  34
[False False False  True  True  True  True  True  True]  -->  5331
[False Fals

*Số trường hợp xảy ra lên đến 101 TH, bây giờ em sẽ đi sắp xếp các trường hợp đó theo 4 đối tượng đã nêu ở phía trên*

##### ***2.1 Số lượng thí sinh thi 7/8/9 môn (không thi 2/1/0 môn)***

In [57]:
print(isNan_score[np.sum(isNan_score, axis=1) == 0])
print(isNan_score[np.sum(isNan_score, axis=1) == 1])
print(isNan_score[np.sum(isNan_score, axis=1) == 2])

[]
[]
[]


- Tất cả đều trả về 0 thí sinh.
- Điều này cũng dễ hiểu vì tổ hợp tự nhiên (lí, hóa, sinh) và xã hội (sử, địa, gdcd) được tổ chức cùng ngày, cùng giờ. Nên không thể có thí sinh nào thi trên 6 môn.

##### ***2.2 Số lượng thí sinh thi 6 môn (không thi 3 môn)***

In [58]:
print(isNan_score[np.sum(isNan_score, axis=1) == 3])
print(np.sum(isNan_counts[np.sum(isNan_score, axis=1) == 3]))

[[False False False False False False  True  True  True]
 [False False False  True  True  True False False False]]
858736


- Có 858,736 thí sinh.
- Đây chính là tổng số tất cả thí sinh thi THPTQG2022
- Bắt buộc thi 2 khối: KHTN và KHXH
    - Toán Văn Anh Lý Hóa Sinh
    - Toán Văn Anh Sử Địa GDCD

##### ***2.3 Số lượng thí sinh thi 5 môn (không thi 4 môn) (thuộc trường hợp giáo dục thường xuyên)***

In [59]:
def asvoid(arr):
    arr = np.ascontiguousarray(arr)
    if np.issubdtype(arr.dtype, np.floating):
        arr += 0.
        
    return arr.view(np.dtype((np.void, arr.dtype.itemsize * arr.shape[-1])))

def inNd(a, b, assume_unique=False):
    return np.in1d(asvoid(a), asvoid(b), assume_unique)

In [60]:
Giao_duc_thuong_xuyen = np.array([[False, False, True, False, False, False, True, True, True],\
                                  [False, False, True, True, True, True, False, False, False]])

isNan_socre_4 = isNan_score[np.sum(isNan_score, axis=1) == 4]
isNan_counts_4 = isNan_counts[np.sum(isNan_score, axis=1) == 4]
gdtx_in_isNanScore4 = inNd(isNan_score[np.sum(isNan_score, axis=1) == 4], Giao_duc_thuong_xuyen)

print(isNan_score[np.sum(isNan_score, axis=1) == 4])
print(np.sum(isNan_counts[np.sum(isNan_score, axis=1) == 4]))

print(isNan_socre_4[gdtx_in_isNanScore4])
print(np.sum(isNan_counts_4[gdtx_in_isNanScore4]))

[[False False False False False  True  True  True  True]
 [False False False False  True False  True  True  True]
 [False False False  True False False  True  True  True]
 [False False False  True  True  True False False  True]
 [False False False  True  True  True False  True False]
 [False False False  True  True  True  True False False]
 [False False  True False False False  True  True  True]
 [False False  True  True  True  True False False False]
 [False  True False False False False  True  True  True]
 [False  True False  True  True  True False False False]
 [ True False False  True  True  True False False False]]
12756
[[False False  True False False False  True  True  True]
 [False False  True  True  True  True False False False]]
9821


- Có tổng số 12756 thí sinh thi 5 môn và trong đó phần lớn thuộc hệ giáo dục thường xuyên (9821 thí sinh)
- Còn lại ta có thể xem như những năm trước đã tốt nghiệp và thi lại lấy điểm cho xét tuyển đại học

##### ***2.4 Số lượng thí sinh thi 3/4/5 môn (không thi 6/5/4 môn) (không tính trường hợp giáo dục thường xuyên)***

In [61]:
print(isNan_score[np.sum(isNan_score, axis=1) == 6])
print(np.sum(isNan_counts[np.sum(isNan_score, axis=1) == 6]))

print(isNan_score[np.sum(isNan_score, axis=1) == 5])
print(np.sum(isNan_counts[np.sum(isNan_score, axis=1) == 5]))

print(isNan_socre_4[~gdtx_in_isNanScore4])
print(np.sum(isNan_counts[np.sum(isNan_score, axis=1) == 4]) - np.sum(isNan_counts_4[gdtx_in_isNanScore4]))

[[False False False  True  True  True  True  True  True]
 [False False  True False  True  True  True  True  True]
 [False False  True  True False  True  True  True  True]
 [False False  True  True  True False  True  True  True]
 [False False  True  True  True  True False  True  True]
 [False False  True  True  True  True  True False  True]
 [False False  True  True  True  True  True  True False]
 [False  True False False  True  True  True  True  True]
 [False  True False  True False  True  True  True  True]
 [False  True False  True  True False  True  True  True]
 [False  True False  True  True  True False  True  True]
 [False  True False  True  True  True  True False  True]
 [False  True False  True  True  True  True  True False]
 [False  True  True False False  True  True  True  True]
 [False  True  True False  True False  True  True  True]
 [False  True  True  True False False  True  True  True]
 [False  True  True  True  True  True False False  True]
 [False  True  True  True  True

- Có 28198 thí sinh thi 3 môn
- Có 94695 thí sinh thi 4 môn
- Có 2935 thí sinh thi 5 môn (không tính trường hợp giáo dục thường xuyên)
- Tổng số các thí sinh trên là những trường hợp thi lại cho xét tuyển đại học

##### ***2.5 Số lượng thí sinh thi 1/2 môn (không thi 8/7 môn)***

In [62]:
print(isNan_score[np.sum(isNan_score, axis=1) == 8])
print(np.sum(isNan_counts[np.sum(isNan_score, axis=1) == 8]))

print(isNan_score[np.sum(isNan_score, axis=1) == 7])
print(np.sum(isNan_counts[np.sum(isNan_score, axis=1) == 7]))

[[False  True  True  True  True  True  True  True  True]
 [ True False  True  True  True  True  True  True  True]
 [ True  True False  True  True  True  True  True  True]
 [ True  True  True  True  True False  True  True  True]
 [ True  True  True  True  True  True False  True  True]
 [ True  True  True  True  True  True  True False  True]
 [ True  True  True  True  True  True  True  True False]]
414
[[False False  True  True  True  True  True  True  True]
 [False  True False  True  True  True  True  True  True]
 [False  True  True False  True  True  True  True  True]
 [False  True  True  True  True False  True  True  True]
 [False  True  True  True  True  True False  True  True]
 [False  True  True  True  True  True  True False  True]
 [False  True  True  True  True  True  True  True False]
 [ True False False  True  True  True  True  True  True]
 [ True False  True  True  True False  True  True  True]
 [ True False  True  True  True  True False  True  True]
 [ True False  True  True 

- Tổng 2 trường hợp có 1050 thí sinh, đây chính là số thí sinh những năm trước chưa tốt nghiệp và bây giờ thi lại để xét tốt nghiệp (không tính thi lại với >= 3 môn vì có thể lấy điểm 3 môn đó xét điểm ĐH)

##### ***2.6 Số lượng thí sinh không thi môn nào***

In [63]:
print(isNan_score[np.sum(isNan_score, axis=1) == 9])
print(np.sum(isNan_counts[np.sum(isNan_score, axis=1) == 9]))

[[ True  True  True  True  True  True  True  True  True]]
6


- Có tổng cộng 6 thí sinh ko có dữ liệu điểm của bất kì môn nào, những trường hợp này sẽ loại bỏ

### **3. Tạo cột phân loại thí sinh**

- Đầu tiên chúng ta sẽ tạo mảng phân loại classify_col có độ dài tương ứng với tổng số thi sinh (sau khi đã drop những thí sinh không thi bất kì môn nào), mảng này gồm những loại sau:
    - Khóa 2022 thi THPTQG và môn tổ hợp là KHTN ('THPT_KHTN')
    - Khóa 2022 thi THPTQG và môn tổ hợp là KHXH ('THPT_KHXH')
    - Khóa 2022 thuộc hệ giáo dục thường xuyên và môn tổ hợp là KHTN ('GDTX_KHTN')
    - Khóa 2022 thuộc hệ giáo dục thường xuyên và môn tổ hợp là KHXH ('GDTX_KHXH')
    - Các khóa trước thi lại cho xét tuyển đại học ('DH_RETEST')
    - Các khóa trước thi lại cho xét tốt nghiệp ('TH_RETEST')
- Sau khi đã phân 6 loại trên vào classify_col ta sẽ tạo cột mới trên df là phan_loai và gán bằng classify_col

In [64]:
# Loại bỏ các dòng có tất cả các môn đều bị thiếu
df = df.drop(np.where(np.sum(np.isnan(df.loc[:, 'toan':]), axis=1) == 9)[0])

# Chuyển dữ liệu về dạng boolean (True: thiếu, False: không thiếu)
isnan_df = np.isnan(df.loc[:, 'toan':])

# x_subject: x là số môn thiếu, x_subject trả về True nếu số môn thiếu bằng x, False nếu không
x_subject = lambda x : np.sum(np.isnan(df.loc[:, 'toan':]), axis=1) == x
classify_col = np.full(df.shape[0], '', dtype='<U9')

# Gán nhãn cho các dòng dữ liệu
classify_col[x_subject(3) * np.isnan(df['lich_su'])] = 'THPT_KHTN'
classify_col[x_subject(3) * np.isnan(df['vat_li'])] = 'THPT_KHXH'
classify_col[inNd(isnan_df, Giao_duc_thuong_xuyen) * np.isnan(df['lich_su'])] = 'GDTX_KHTN'
classify_col[inNd(isnan_df, Giao_duc_thuong_xuyen) * np.isnan(df['vat_li'])] = 'GDTX_KHXH'
classify_col[x_subject(6) + x_subject(5) + (x_subject(4) * (~inNd(isnan_df, Giao_duc_thuong_xuyen)))] = 'DH_RETEST'
classify_col[x_subject(8) + x_subject(7)] = 'TH_RETEST'

df['phan_loai'] = classify_col

- Có thể thấy số lượng thí sinh từng loại hoàn toàn khớp với phần xác định các trường hợp

In [65]:
print(np.sum(df['phan_loai'] == 'THPT_KHTN'))
print(np.sum(df['phan_loai'] == 'THPT_KHXH'))
print(np.sum(df['phan_loai'] == 'GDTX_KHTN'))
print(np.sum(df['phan_loai'] == 'GDTX_KHXH'))
print(np.sum(df['phan_loai'] == 'DH_RETEST'))
print(np.sum(df['phan_loai'] == 'TH_RETEST'))
df

309162
549574
6100
3721
125828
1050


,cum,sbd,toan,ngu_van,ngoai_ngu,vat_li,hoa_hoc,sinh_hoc,lich_su,dia_ly,gdcd,phan_loai
0,1,1000001,3.6,5.00,4.0,NaN,NaN,NaN,2.75,6.00,8.75,THPT_KHXH
1,1,1000002,8.4,6.75,7.6,NaN,NaN,NaN,8.50,7.50,8.25,THPT_KHXH
2,1,1000003,5.8,7.50,5.0,NaN,NaN,NaN,7.25,5.50,8.75,THPT_KHXH
3,1,1000004,7.4,7.50,8.6,NaN,NaN,NaN,7.50,6.50,7.50,THPT_KHXH
4,1,1000005,7.2,8.50,9.0,NaN,NaN,NaN,8.00,8.50,8.25,THPT_KHXH
...,...,...,...,...,...,...,...,...,...,...,...,...
995436,64,64006584,8.4,6.75,4.6,NaN,NaN,NaN,6.50,6.75,9.00,THPT_KHXH
995437,64,64006585,5.6,6.50,2.8,NaN,NaN,NaN,6.25,6.75,8.50,THPT_KHXH
995438,64,64006586,5.8,6.00,6.6,NaN,NaN,NaN,7.25,8.00,8.00,THPT_KHXH
995439,64,64006587,7.6,6.75,7.0,NaN,NaN,NaN,8.75,7.25,9.75,THPT_KHXH


### **4. Ghi kết quả phân loại và file mới**

- Bây giờ ta sẽ lưu kết quả vừa phân loại được vào file mới và khi phân tích ta sẽ đọc file này

In [68]:
df.to_csv('new_data.csv',index=False)